In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/gemstone.csv')

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [13]:
df = df.drop(labels=['id'],axis=1)

In [14]:
#Seperation of Independent and Dependent Variables
#Independent Variable
X = df.drop(labels=['price'],axis=1)

In [15]:
#Dependent variable
Y = df[['price']]

In [16]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [17]:
Y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


Now we need to Scale the Data which is continuous and Ordinal-encode data which is Ordinal (categorical but ranked)

In [18]:
numerical_columns = X.select_dtypes(exclude='object').columns
categorical_columns = X.select_dtypes(include='object').columns

In [19]:
numerical_columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [20]:
categorical_columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [21]:
#Give ranking to each Ordinal Variable i.e cut, color, clarity

In [22]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [24]:
from sklearn.impute import SimpleImputer #To handle missing values
from sklearn.preprocessing import StandardScaler #to reach global minima in Linear Regression i.e Feature Scaling
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding since there are categorical variables with rank


In [25]:
#Create Pipeline
from sklearn.pipeline import Pipeline
#To group pipeling together
from sklearn.compose import ColumnTransformer

In [33]:
#Numerical Pipeling
'''
1) Handle Missing values
2) Scaling
'''
numerical_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')), #Median since there are outliers
    ('scaler',StandardScaler())
    ]
)

#Categorical Piplepine
'''
1) Handle Missing values
2) Ordinal Encoding
3) Scaling
'''
categorical_pipeline = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
    ('scaler',StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ('numerical_pipeline',numerical_pipeline,numerical_columns),
    ('categorical_pipeline',categorical_pipeline,categorical_columns)
])

In [40]:
#Train test spli
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=30)

In [41]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [43]:
X_test.head()

,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,-0.132136,-0.935071,0.019720
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,-1.138347,0.912774,-0.646786
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,0.874076,0.912774,2.685743
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,-0.132136,0.296826,0.019720
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,-0.132136,-0.935071,0.686225


In [44]:
#Model Training
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [45]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [46]:
regression.coef_

array([[ 6433.66003594,  -132.75843566,   -70.42922179, -1720.30971463,
         -499.29302619,   -63.39317848,    72.44537247,  -460.41604642,
          650.76431652]])

In [47]:
regression.intercept_

array([3970.76628955])

In [48]:
import numpy as np
def evaluate(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_square = r2_score(true,predicted)
    return mae,mse,rmse,r2_square

In [51]:
#Training of Different models
models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    #Predictions on test data
    y_pred = model.predict(X_test)

    mae,mse,rmse,r2_square = evaluate(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print('MAE:- ', mae)
    print('MSE:- ', mse)
    print('RMSE:- ',rmse)
    print('R2_Score',r2_square*100)

    r2_list.append(r2_square)

    print("*"*40)
    print('\n')

LinearRegression
Model Training Performance
MAE:-  674.0255115796832
MSE:-  1028002.7598132559
RMSE:-  1013.9047094344004
R2_Score 93.68908248567512
****************************************


Lasso
Model Training Performance
MAE:-  675.0716923362162
MSE:-  1027949.4559693959
RMSE:-  1013.8784226767013
R2_Score 93.68940971841704
****************************************


Ridge
Model Training Performance
MAE:-  674.0555800798198
MSE:-  1028005.2293677676
RMSE:-  1013.9059272771649
R2_Score 93.68906732505937
****************************************


ElasticNet
Model Training Performance
MAE:-  1060.7368759154729
MSE:-  2351365.3822896425
RMSE:-  1533.416245606405
R2_Score 85.56494831165182
****************************************


